<a href="https://colab.research.google.com/github/DEEP-CGPS/FRUIT-16K/blob/master/PruningRedes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torch import nn
import torch.nn.utils.prune as prune
from __future__ import print_function, division
import torch.utils.data as data
import torchvision
from torchvision import transforms
import torch
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame
import matplotlib.font_manager as fm
from matplotlib.collections import QuadMesh
import seaborn as sn
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes=13):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = self.classifier(x)
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
from torchsummary import summary
model=Net().to(device)
print(summary(model,(3,224,224)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 55, 55]          23,296
              ReLU-2           [-1, 64, 55, 55]               0
         MaxPool2d-3           [-1, 64, 27, 27]               0
            Conv2d-4          [-1, 192, 27, 27]         307,392
              ReLU-5          [-1, 192, 27, 27]               0
         MaxPool2d-6          [-1, 192, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         663,936
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 256, 13, 13]         884,992
             ReLU-10          [-1, 256, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         590,080
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
AdaptiveAvgPool2d-14            [-1, 25

In [ ]:
number=31

In [ ]:
new_model=Net()

In [ ]:
new_model=torch.load('ModelUTBP.pth')
new_model.eval()

FileNotFoundError: ignored

In [ ]:
def RESconv(module):
  clist=[]
  dw=len(module.weight)
  for i in range(0,len(module.weight)): 
    x=module.weight[i]
    y=module.bias[i]
    value = 0
    x = x[x.sum()!=value]
    a,b,c,d=x.size()
    if a==0:
      clist.append(i)
      dw=dw-1
      module.bias[i]=0
    if i>=1 and a!=0:
      x=torch.cat([xan,x])
      xan=x
    if i==0:
      xan=x
  value=0
  y=module.bias[:]
  y = y[y!=value]
  return xan,y,dw,clist

def CerosConv(module,clist,dw):
  module.in_channels=dw
  for i in range(0,len(module.weight[:])):
      y=module.weight[i][:]
      for j in clist:
          y[j]=0
      for k in range(0,len(y)): 
          x=y[k]
          value = 0
          x = x[x.sum()!=value]
          a,b,c=x.size()
          if k>=1 and a!=0:
            x=torch.cat([xan,x])
            xan=x
          if k==0:
            xan=x
      if i==0:    
          nx=xan.unsqueeze(0)
      else:
          nx=torch.cat([nx,xan.unsqueeze(0)])
  return nx

def RESFC(module):
  clist=[]
  dw=len(module.weight)
  for i in range(0,len(module.weight)): 
    x=module.weight[i]
    y=module.bias[i]
    value = 0
    x = x[x.sum()!=value]
    a,b=x.size()
    if a==0:
      clist.append(i)
      dw=dw-1
      module.bias[i]=0
    if i==0:    
      nx=x
    else:
      nx=torch.cat([nx,x])
  value=0
  y=module.bias[:]
  y = y[y!=value]
  return nx,y,dw,clist

def CerosFC(module,clist,imdfc1,fc1):
  nclist=[]
  if fc1==1:
    dim=imdfc1*imdfc1
    for i in range(0,len(clist)):
      inp=clist[i]*dim
      enp=inp+dim
      listprev=list(range(inp,enp))
      for j in listprev:
        nclist.append(j)
  else:
    nclist=clist
  for i in range(0,len(module.weight[:])):
    y=module.weight[i][:]
    for j in nclist:
      y[j]=0
    x=y
    value = 0
    x = x[x !=value]
    if i==0:    
        nx=x.unsqueeze(0)
    else:
        nx=torch.cat([nx,x.unsqueeze(0)])
  return nx

In [ ]:
imdfc1=6#se debe conocer apriori, a futuro se podria hacer una face de foward hasta antes de la fc1 para calcularlo de forma automatico
dl=0
dln=0
cont=0
for name, module in new_model.named_modules():
  dl+=1
print(dl)

pp=0.8
for name, module in new_model.named_modules():
  if isinstance(module, torch.nn.Conv2d):
    if cont==1:
      nx=CerosConv(module,clist,dw)
      module.weight=nn.Parameter(nx) 
    clist=[]    
    prune.ln_structured(module, name="weight", amount=pp, n=2, dim=0)
    prune.remove(module, 'weight')
    print(list(module.named_parameters()))
    xan,y,dw,clist=RESconv(module)
    module.out_channels=dw
    module.weight=nn.Parameter(xan)
    module.bias=nn.Parameter(y)
    cont=1
    print("entrooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooodadawdawda")
  if isinstance(module, torch.nn.Linear) and cont==1:
    module.in_features=dw*imdfc1*imdfc1
    nx=CerosFC(module,clist,imdfc1,1)
    module.weight=nn.Parameter(nx)
    cont=2

cont=0
for name, module in new_model.named_modules():
  dln+=1
  if isinstance(module, torch.nn.Linear):
    if cont==1:
      module.in_features=dw
      nx=CerosFC(module,clist,imdfc1,0)
      module.weight=nn.Parameter(nx)
    if dln<dl:
      prune.ln_structured(module, name="weight", amount=pp, n=2, dim=0)
      prune.remove(module, 'weight')
      nx,y,dw,clist=RESFC(module)
      module.out_features=dw
      module.weight=nn.Parameter(nx)
      module.bias=nn.Parameter(y)
      cont=1

24
[('bias', Parameter containing:
tensor([-0.0086, -0.0333, -0.0214,  0.0360, -0.0085,  0.0470, -0.0257,  0.0223,
        -0.0113, -0.0013,  0.0120,  0.0459,  0.0156, -0.0492, -0.0162, -0.0028,
        -0.0260, -0.0258,  0.0303,  0.0517,  0.0393, -0.0178, -0.0409,  0.0182,
        -0.0289,  0.0460, -0.0084, -0.0447, -0.0472, -0.0509, -0.0518,  0.0277,
        -0.0016,  0.0208, -0.0161,  0.0180, -0.0226,  0.0027, -0.0193,  0.0177,
         0.0145,  0.0374,  0.0503,  0.0137, -0.0500,  0.0261,  0.0246,  0.0006,
        -0.0415, -0.0059, -0.0472, -0.0512,  0.0117, -0.0204,  0.0218,  0.0501,
        -0.0148, -0.0040,  0.0109,  0.0014,  0.0480, -0.0043,  0.0320, -0.0513],
       requires_grad=True)), ('weight', Parameter containing:
tensor([[[[0., 0., -0.,  ..., 0., -0., 0.],
          [0., 0., 0.,  ..., -0., 0., -0.],
          [0., -0., 0.,  ..., 0., -0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., -0., 0.],
          [-0., -0., 0.,  ..., 0., -0., -0.],
          [0., 0., 0.,  ..

KeyboardInterrupt: ignored

In [ ]:
from torchsummary import summary
print(summary(new_model,(3,224,224)))

In [ ]:
torch.save(new_model,str(int(pp*100))+'-'+'ModeloAPBT.pth')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
